In [1]:
import os
import polars as pl
import altair as alt

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [2]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

## Rychlé sčoty

In [4]:
df.group_by_dynamic(index_column="datum",every="1w").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True).head(10)

datum,pocet_slov
date,u32
2011-10-31,507439
2023-02-27,478349
2023-07-10,310767
2022-02-14,290641
2024-01-22,272956
2024-05-27,267470
2024-01-15,264061
2024-10-21,261888
2023-09-04,252238


In [5]:
df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True).head(10)

datum,pocet_slov
date,u32
2023-03-03,131371
2025-04-16,128315
2023-10-18,126715
2011-11-02,121073
2022-02-02,120047
2024-01-24,117239
2023-03-01,116301
2023-03-02,113215
2023-09-06,110182


In [6]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1300
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [7]:
graf_obdobi_93 = alt.Chart(
    df.group_by("obdobi").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("obdobi:N"),
    alt.Y("pocet_slov:Q")
)

graf_obdobi_93

alt.Chart(...)

In [8]:
import os
try:
    os.makedirs("grafy")
except:
    pass

In [9]:
graf_obdobi_93.save(os.path.join("grafy","graf_obdobi_93.png"))

In [10]:
slov_na_den = df.group_by("obdobi").agg(pl.col('datum').max()).rename({'datum':'do'}).join(
    df.group_by("obdobi").agg(pl.col('datum').min()).rename({'datum':'od'}),
    how='left',on='obdobi'
).join(
    df.group_by("obdobi").agg(pl.col("pocet_slov").sum()),
    how='left',on='obdobi'
).with_columns(
    (pl.col('do') - pl.col('od')).alias('trvani').dt.total_days()
).with_columns(
    (pl.col('pocet_slov') / pl.col('trvani')).alias('slov_na_den')
)

In [11]:
alt.Chart(
    slov_na_den.to_pandas()
).mark_bar(
).encode(
    alt.X("obdobi:N"),
    alt.Y("slov_na_den:Q")
)

alt.Chart(...)

In [12]:
df.filter(pl.col('obdobi') == 1996).filter(pl.col('datum') > pl.date(2004, 1, 1))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32


## Pro srovnání: Babička

In [14]:
import requests
from bs4 import BeautifulSoup

In [15]:
babicka = requests.get("https://web2.mlp.cz/koweb/00/03/34/76/44/babicka.html").text
babicka = BeautifulSoup(babicka).text.split("Dávno, dávno již tomu")[1].split("Vydala Městská knihovna v Praze")[0]

In [16]:
babicka.count(" ")

67169

In [17]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() / babicka.count(" ")

232.08657267489468

## Dlouhé x krátké projevy

Cílem studie bylo najít krátký a jednoduchý test k měření psychomotorického tempa, který by se dal použít v klinické praxi. Metodika: Bylo vyšetřeno 63 zdravých dobrovolníků průměrného stáří 28,6 ± 12,5 let (rozsah 15–71 let). Byl měřen počet slov za jednu minutu spontánního monologu (měřítko psychomotorického tempa), čas nutný k vyjmenování 12 zvířat (zkrácená verze sémantické fluence), 12 slov začínajících na písmeno K (zkrácená verze fonemické fluence), čas potřebný k vyjmenování měsíců v roce, k hlasitému počítání od 101 do 110 a jiné krátké testy. Výsledky: Během spontánního monologu bylo zaznamenáno 105,7 ± 38,7 slov za minutu. Byla nalezena negativní korelace mezi spontánní řečí a rychlostí počítání (r = -0,69; p < 0,001). Vyjmenování 12 měsíců bylo více než 3× rychlejší než vyjmenování 12 zvířat a 7× rychlejší než vyjmenování 12 slov na K.

Zdroj: [PSYCHOMOTORICKÉ TEMPO, RYCHLOST ŘEČI A MYŠLENÍ RYCHLOST ŘEČI A MYŠLENÍ](https://www.psychiatriepropraxi.cz/pdfs/psy/2007/05/06.pdf)

Bacha: i dlouhý projev může být přerušený krátkou vsuvkou.

In [21]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() 

15589023

In [22]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() / (100 * 60 * 40)

64.9542625

In [23]:
alt.Chart(
    df.filter(pl.col("pocet_slov") <= 100).group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [24]:
alt.Chart(
    df.filter(pl.col("pocet_slov") >= 1000).group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

## Kolik toho není přepsáno?

In [26]:
neprepsane = pl.read_parquet("data/neprepsane_schuze.parquet").filter(pl.col('komora') == 'ps')

In [27]:
neprepsane

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,i32,i32,i32,str,date,str,str,bool,bool
"""ps""",2013,9,9,"""2013ps_stenprot_009schuz_s009009.htm""",null,null,null,false,null
"""ps""",2013,10,306,"""2013ps_stenprot_010schuz_s010306.htm""",null,null,null,false,null
"""ps""",2013,23,944,"""2013ps_stenprot_023schuz_s0231944.htm""",null,null,null,false,null
"""ps""",2013,23,945,"""2013ps_stenprot_023schuz_s0231945.htm""",null,null,null,false,null
"""ps""",2013,23,946,"""2013ps_stenprot_023schuz_s0231946.htm""",null,null,null,false,null
"""ps""",2013,23,947,"""2013ps_stenprot_023schuz_s0231947.htm""",null,null,null,false,null
"""ps""",2013,23,948,"""2013ps_stenprot_023schuz_s0231948.htm""",null,null,null,false,null
"""ps""",2013,23,949,"""2013ps_stenprot_023schuz_s0231949.htm""",null,null,null,false,null
"""ps""",2013,23,950,"""2013ps_stenprot_023schuz_s0231950.htm""",null,null,null,false,null


In [28]:
df.group_by("obdobi").agg(pl.col("soubor").unique().len()).sort(by="soubor").rename({"soubor":"prepsanych"}).join(
    neprepsane.group_by("obdobi").agg(pl.col("soubor").unique().len()).sort(by="obdobi").rename({'soubor':"neprepsanych"}),
    how='left', on='obdobi'
).with_columns(
    (pl.col('prepsanych') + pl.col('neprepsanych')).alias('celkem')
).with_columns(
    (pl.col('neprepsanych') / pl.col('celkem')).alias('podil_neprepsanych')
)

obdobi,prepsanych,neprepsanych,celkem,podil_neprepsanych
i32,u32,u32,u32,f64
1993,1149,null,null,null
1996,1932,null,null,null
2010,6508,null,null,null
2006,7623,null,null,null
1998,9904,null,null,null
2002,10183,null,null,null
2013,10461,721,11182,0.064479
2017,11962,281,12243,0.022952
2021,13859,38,13897,0.002734


## Přibudou ještě?

In [30]:
df.filter(pl.col("obdobi") == 2017).filter(pl.col("datum") > pl.date(2021,9,1)).select(pl.col("pocet_slov")).sum()

pocet_slov
u32
214637


In [31]:
df.filter(pl.col("obdobi") == 2017).select(pl.col("datum").max())

datum
date
2021-10-05
